In [40]:
import pandas as pd 
import numpy as np

In [41]:
df = pd.read_csv('Levels_Fyi_Salary_Data.csv')

In [42]:
df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33:27,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11:29,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53:57,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23:14,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000.0,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58:51,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0.0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [43]:
df.columns

Index(['timestamp', 'company', 'level', 'title', 'totalyearlycompensation',
       'location', 'yearsofexperience', 'yearsatcompany', 'tag', 'basesalary',
       'stockgrantvalue', 'bonus', 'gender', 'otherdetails', 'cityid', 'dmaid',
       'rowNumber', 'Masters_Degree', 'Bachelors_Degree', 'Doctorate_Degree',
       'Highschool', 'Some_College', 'Race_Asian', 'Race_White',
       'Race_Two_Or_More', 'Race_Black', 'Race_Hispanic', 'Race', 'Education'],
      dtype='object')

In [44]:
missing_cols = []
for col in df.columns:
    if df[col].isnull().values.any():
        num = df[col].isnull().sum()
        missing_cols.append([col, num])

In [45]:
missing_cols

[['company', 5],
 ['level', 119],
 ['tag', 854],
 ['gender', 19540],
 ['otherdetails', 22505],
 ['dmaid', 2],
 ['Race', 40215],
 ['Education', 32272]]

In [46]:
df = df.drop(['Race', 'Education', 'otherdetails', 'level', 'tag', 'dmaid'], axis = 1)

In [49]:
df.dropna(inplace = True)
df.reset_index(drop=True, inplace=True) 

In [51]:
len(df)

43098

In [55]:
df.columns

Index(['timestamp', 'company', 'title', 'totalyearlycompensation', 'location',
       'yearsofexperience', 'yearsatcompany', 'basesalary', 'stockgrantvalue',
       'bonus', 'gender', 'cityid', 'rowNumber', 'Masters_Degree',
       'Bachelors_Degree', 'Doctorate_Degree', 'Highschool', 'Some_College',
       'Race_Asian', 'Race_White', 'Race_Two_Or_More', 'Race_Black',
       'Race_Hispanic'],
      dtype='object')

In [59]:
df.groupby(by = 'company')['totalyearlycompensation'].mean().to_frame().sort_values(by = 'totalyearlycompensation')

,totalyearlycompensation
company,
Bukalapak,1.260000e+04
TATA Consultancy Services,1.300000e+04
Nice,1.400000e+04
verkada,1.500000e+04
Refinitiv,1.500000e+04
...,...
UBER,5.500000e+05
PDT Partners,5.583333e+05
Doordash,5.620000e+05


In [60]:
df['totalyearlycompensation'].describe()

count    4.309800e+04
mean     2.098304e+05
std      1.347862e+05
min      1.000000e+04
25%      1.310000e+05
50%      1.840000e+05
75%      2.550000e+05
max      4.980000e+06
Name: totalyearlycompensation, dtype: float64

In [64]:
df[df['totalyearlycompensation'] > 4000000]

,timestamp,company,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,...,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic
42699,8/20/2019 15:35:10,Microsoft,Software Engineering Manager,4950000,"Redmond, WA",24.0,8.0,850000.0,3750.0,350000.0,...,0,0,0,0,0,0,0,0,0,0
42710,3/9/2021 17:03:07,Google,Product Manager,4500000,"San Francisco, CA",20.0,14.0,450000.0,3700.0,350000.0,...,1,0,0,0,0,1,0,0,0,0
42714,7/30/2021 22:23:24,Facebook,Product Manager,4980000,"Menlo Park, CA",17.0,0.0,380000.0,4400.0,200000.0,...,1,0,0,0,0,1,0,0,0,0


In [65]:
df.location.value_counts()

Seattle, WA                          5800
San Francisco, CA                    4319
New York, NY                         3102
Redmond, WA                          1838
Sunnyvale, CA                        1528
                                     ... 
Graz, ST, Austria                       1
Trondheim, ST, Norway                   1
Mandaluyong City, MM, Philippines       1
Elk Grove, CA                           1
Hilbert, WI                             1
Name: location, Length: 936, dtype: int64